
### Shehroz Sohail

![alt text](https://cdn.wccftech.com/wp-content/uploads/2021/02/twitter-for-iOS.jpg)

### Objective:
+ We were hired by twitter to come up with a default sentiment analysis model which can be integrated into their API and will make analysis sentiment throug hthe twitter API much much easier. For the purpose of this research we were given access to their API network and selected to analyze the current situation of Russia-Ukraine. 
+ Extracted 50,000 tweets with the Russia hashtag and 50,000 tweets with the Ukraine hashtag. This ensures that both of the sides are represented and no unnecessary bias is introduced into the model.

### Analysis.
+ For this purpose we will be comparing two techniques of sentiment analysis. The first one is the vader sentiment analysis in the NLTK library. The vader sentiment has a dictionary of positive words and negative words in it and simply calculates the sentiment by counting how many time a positive or a negative word appears in the snetence. Then it calcuates probabilities and hence assigns labels.
+ The other technique is the neural network whcih unlike the previous approach which considers each word to be independant from the context of the sentence, this model considers the previous word while assigning labels to the sentence.

### Approach.
+ After extracting tweets using the twitter APi. we will first split our dataset into train, validate and test set.
+ we will apply vader sentiment on all of the given datasets and save them for later use. 
+ The train and validate set will be used to create our model (supervised learning using labels from the vader sentiment)
+ I will then label the test set physically and use it as  a basis to check which model performs better.
+ Both the vader sentiment and model's accuracy scores will be calculated on the physically labeled test set to see which one performs better.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import string
import random
# import tweepy
# from tweepy import *
import datetime

#NLTK
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorboard.plugins import projector
import IPython
from sklearn.model_selection import train_test_split
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#### Importing the Necessary Keys/Secrets to work with twitter API.
+ All of the codes have been changed and will not work in a new notebook unless new codes are inserted

In [ ]:
consumer_key = 'yALdWFxZ0DVDSDGYczMzakfVn'
consumer_secret = '4YgHvQz9rDjMv5Btq9b9ygimkkKZb0jtLFwoYSg5eqjMaSf0Vo'
access_key= '903728159010508801-Fg8gJnZ6vLzRkYUWJgNf2L4EpJcXgIv'
access_secret = 'DNMq4P2BRSsqYszuwlL2iaoGVmujF5nUZoOMa8GKvUEAV'
Bearer_token='AAAAAAAAAAAAAAAAAAAAAHI8aQEAAAAA8QCbMBdiz4xpnNO3XAu2ikD77SI%3D7kzqydWWB7iWtMhqZpMblJg3ya81hqe9GjXh7KDh2xpFxTuzzG'



auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
 
api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
today = datetime.date.today()
yesterday= today - datetime.timedelta(days=300)
keywords="#Russia since:" + str(yesterday)+ " until:" + str(today)
# keywords = 'russia'
limit=50000

tweets = tweepy.Cursor(api.search_tweets,since_id="2022-01-01", q=keywords, count=300, tweet_mode='extended',lang="en").items(limit)

##### A tweepy Cursor is an item iterator that contains JSON files at each iteration (till the limit is reached)
+ Displaying the first json output and extracting information that is needed. 

In [ ]:
for tweet in tweets:
    x=tweet._json
    if len(x)!=0:
        break
x

Rate limit reached. Sleeping for: 430


{'created_at': 'Sat Mar 19 23:59:58 +0000 2022',
 'id': 1505333290990940160,
 'id_str': '1505333290990940160',
 'full_text': 'RT @NewUkraineNews: Female #Ukrainian soldier with a #Javelin anti tank missile launcher, near #Kiev.\n#Ukraine #UkraineRussiaWar #Russia #P…',
 'truncated': False,
 'display_text_range': [0, 140],
 'entities': {'hashtags': [{'text': 'Ukrainian', 'indices': [27, 37]},
   {'text': 'Javelin', 'indices': [53, 61]},
   {'text': 'Kiev', 'indices': [95, 100]},
   {'text': 'Ukraine', 'indices': [102, 110]},
   {'text': 'UkraineRussiaWar', 'indices': [111, 128]},
   {'text': 'Russia', 'indices': [129, 136]}],
  'symbols': [],
  'user_mentions': [{'screen_name': 'NewUkraineNews',
    'name': 'Ukraine News Now',
    'id': 1496723055506169856,
    'id_str': '1496723055506169856',
    'indices': [3, 18]}],
  'urls': []},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPho

##### Picking only the necesssary information from the JSON files and storing them in a list for further analysis

In [ ]:
columns = ['User', 'Tweet','Date','location']
data = []

for tweet in tweets:
    data.append([tweet.full_text])

# Russia_df = pd.DataFrame(data, columns=columns)
# Russia_df['label']='Russia'
# Russia_df.to_csv('Russia_50,000.csv')
# print(len(Russia_df))
# Russia_df

Rate limit reached. Sleeping for: 824
Rate limit reached. Sleeping for: 820


#### Repeating the above process again for tweets with the Ukraine hashtag

In [ ]:
keywords="#Ukraine since:" + str(yesterday)+ " until:" + str(today)
limit=50000
tweets = tweepy.Cursor(api.search_tweets,since_id="2022-01-01", q=keywords, count=300, tweet_mode='extended',lang="en").items(limit)

#######################################################################
# columns = ['User', 'Tweet','Date','location']
data_1 = []

for tweet in tweets:
    data_1.append([tweet.full_text])

# Ukraine_df = pd.DataFrame(data_1, columns=columns)
# Ukraine_df['label']='Ukraine'
# Ukraine_df.to_csv('Ukraine_50,000.csv')

Rate limit reached. Sleeping for: 821
Rate limit reached. Sleeping for: 820
Rate limit reached. Sleeping for: 822


### Importing the created datasets


In [3]:
df_Russia=pd.read_csv(r'Russia_50,000.csv')
df_Ukraine=pd.read_csv(r'Ukraine_50,000.csv')
df=df_Russia.append(df_Ukraine , ignore_index=True)
df = df.sample(frac=1,random_state=218).reset_index(drop=True)
df['label']=df['label'].where(df['label']=='Russia',0)
df['label']=df['label'].where(df['label']==0,1)
# df.isna().sum()
# df_Ukraine
df=df.Tweet
df.head()
testset=df[:50]
testset_labels=np.array([0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0])
df=df[50:].reset_index(drop=True)

C:\Users\shzso\AppData\Local\Temp\ipykernel_27148\2503116619.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df_Russia.append(df_Ukraine , ignore_index=True)


In [4]:
vs=[]
analyzer = SentimentIntensityAnalyzer()
for i in range(len(df)):
    temp=(analyzer.polarity_scores(df[i]))
    vs.append(temp['compound'])

for i in range(len(vs)):
    if vs[i]>=0:
        vs[i]=1
    if vs[i] < 0:
        vs[i]=0
# df[0]
# temp=analyzer.polarity_scores(df[2])
vs

[1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,


In [5]:
training_sentences_unprocessed, testing_sentences_unprocessed, training_labels, testing_labels=train_test_split(df, vs, test_size=0.33, random_state=42)

In [6]:
# Processing the tweets
def process_tweet(tweet):
    vocab_size = 10000
    oov_tok = '<OOV>'
 
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove twitter abbreviations
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweet

In [7]:
training_sentences = []
for i, tweet in enumerate(training_sentences_unprocessed): 
    training_sentences.append(process_tweet(tweet))

testing_sentences = []
for i, tweet in enumerate(testing_sentences_unprocessed): 
    testing_sentences.append(process_tweet(tweet))

# Convert labels lists to numpy array
training_labels_final = []
testing_labels_final = []

for i in training_labels:
    training_labels_final.append(tf.cast(i, tf.int32))

for i in testing_labels:
    testing_labels_final.append(tf.cast(i, tf.int32))


training_labels_final = np.array(training_labels_final)
testing_labels_final = np.array(testing_labels_final)

In [8]:
# Parameters

vocab_size = 7000
max_length = 120
embedding_dim = 16
trunc_type='post'
oov_tok = "<OOV>"

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize the Tokenizer class
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)

# Generate the word index dictionary for the training sentences
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

# Generate and pad the training sequences
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

# Generate and pad the test sequences
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [10]:
import tensorflow as tf

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.LSTM(120),
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Setup the training parameters
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           112000    
                                                                 
 lstm (LSTM)                 (None, 120)               65760     
                                                                 
 flatten (Flatten)           (None, 120)               0         
                                                                 
 dense (Dense)               (None, 6)                 726       
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 178,493
Trainable params: 178,493
Non-trainable params: 0
_________________________________________________________________


In [11]:
num_epochs = 5

# Train the model
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))
model.save('senitment_model')

Epoch 1/5
2093/2093 [==============================] - 32s 14ms/step - loss: 0.2310 - accuracy: 0.8989 - val_loss: 0.1460 - val_accuracy: 0.9444
Epoch 2/5
2093/2093 [==============================] - 29s 14ms/step - loss: 0.1105 - accuracy: 0.9581 - val_loss: 0.1223 - val_accuracy: 0.9543
Epoch 3/5
2093/2093 [==============================] - 29s 14ms/step - loss: 0.0796 - accuracy: 0.9699 - val_loss: 0.1266 - val_accuracy: 0.9569
Epoch 4/5
2093/2093 [==============================] - 33s 16ms/step - loss: 0.0587 - accuracy: 0.9774 - val_loss: 0.1286 - val_accuracy: 0.9577
Epoch 5/5
2093/2093 [==============================] - 35s 17ms/step - loss: 0.0450 - accuracy: 0.9826 - val_loss: 0.1414 - val_accuracy: 0.9579


INFO:tensorflow:Assets written to: senitment_model\assets


INFO:tensorflow:Assets written to: senitment_model\assets


In [12]:
# Get the embedding layer from the model (i.e. first layer)
embedding_layer = model.layers[0]

# Get the weights of the embedding layer
embedding_weights = embedding_layer.get_weights()[0]

# Print the shape. Expected is (vocab_size, embedding_dim)
print(embedding_weights.shape) 

(7000, 16)


In [13]:
# Get the index-word dictionary
reverse_word_index = tokenizer.index_word

In [14]:
import io

# Open writeable files
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

# Initialize the loop. Start counting at `1` because `0` is just for the padding
for word_num in range(1, vocab_size):

  # Get the word associated at the current index
  word_name = reverse_word_index[word_num]

  # Get the embedding weights associated with the current index
  word_embedding = embedding_weights[word_num]

  # Write the word name
  out_m.write(word_name + "\n")

  # Write the word embedding
  out_v.write('\t'.join([str(x) for x in word_embedding]) + "\n")

# Close the files
out_v.close()
out_m.close()

In [15]:
url = 'https://projector.tensorflow.org/'

IPython.display.IFrame(url, width=1333, height=900)

### Final testing to see which model performed better

In [16]:
def get_scores (testset,testset_labels):
    final_test_sentences = []
    for i, tweet in enumerate(testset): 
        final_test_sentences.append(process_tweet(tweet[0]))


    testing_sequences = tokenizer.texts_to_sequences(final_test_sentences)
    final_test_padded = pad_sequences(testing_sequences,maxlen=max_length)

    prediction=(model.predict(final_test_padded).round()).flatten()
    prediction=prediction.astype(int)

    vs_test=[]
    analyzer = SentimentIntensityAnalyzer()
    for i in range(50):
        temp=(analyzer.polarity_scores(testset.iloc[i]))
        vs_test.append(temp['compound'])

    for i in range(len(vs_test)):
        if vs_test[i]>=0:
            vs_test[i]=1
        if vs_test[i] < 0:
            vs_test[i]=0
    vs_test=np.array(vs_test)

    # pred_accuracy=((prediction==testset_labels))#.sum()/700)*100
    # vs_accuracy=((vs_test==testset_labels))#.sum()/700)*100
    return prediction,vs_test

In [17]:

prediction, vs_test=get_scores(testset=testset,testset_labels=testing_labels)
model_prediction_acc=(testset_labels==prediction).sum()/len(testset_labels)
vader_prediction_acc=(testset_labels==vs_test).sum()/len(testset_labels)
print(f'neural network acc: {model_prediction_acc} vader sentiment accuracy: {vader_prediction_acc}')

neural network acc: 0.66 vader sentiment accuracy: 0.62


### Conclusion
+ As we can see from above we get a higher accuracy from the LSTM model and a lower accuracy from the simpler model
+ one can assume the difference in accuracy can be seen because our model does not assume the independance of words and instead treats the word in the context of the sentence.
+ Although there is an increase in accufracy this cannot be treated as conclusive evidence for this and we need to perform further tests over different models to reach this conclusion
+ For th emeantime the call to action for twitter is to create a deep learning neural network to suggest a sentiment analysis for its users instead of relying on the simpler model/]